In [1]:
'''
Data:
https://www.kaggle.com/datasets/grassknoted/asl-alphabet/download?datasetVersionNumber=1
'''


import tensorflow as tf
import keras
from keras import layers,losses,optimizers
import matplotlib.pyplot as plt


# HyperParameters

In [2]:
TRAINPATH="D:\\df\\ai\\asl\\asl_alphabet_train\\asl_alphabet_train\\"
TESTPATH="D:\\df\\ai\\asl\\asl_alphabet_test\\asl_alphabet_test\\"
IMAGE_WIDTH=IMAGE_HEIGHT=224
BATCH_SIZE=64
NUM_EPOCHS=5
LR=3e-2

# Build Data pipeline

## Manage Data

In [3]:
import os
import pandas as pd
import numpy as np

classes=os.listdir(TRAINPATH)
print(classes)
images=[]
outputs=[]

for letter in classes:
    for image in os.listdir(TRAINPATH+letter):
        images.append(TRAINPATH+letter+'\\'+image)
        outputs.append(letter)
trainframe=pd.DataFrame({"images":images,"output":outputs})
images.clear()
outputs.clear()

for test in os.listdir(TESTPATH):
    images.append(TESTPATH+test)
    outputs.append(test[0])
testframe=pd.DataFrame({"images":images,"output":outputs})

trainframe.head(10)


['A', 'B', 'C', 'D', 'del', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'nothing', 'O', 'P', 'Q', 'R', 'S', 'space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


,images,output
0,D:\df\ai\asl\asl_alphabet_train\asl_alphabet_t...,A
1,D:\df\ai\asl\asl_alphabet_train\asl_alphabet_t...,A
2,D:\df\ai\asl\asl_alphabet_train\asl_alphabet_t...,A
3,D:\df\ai\asl\asl_alphabet_train\asl_alphabet_t...,A
4,D:\df\ai\asl\asl_alphabet_train\asl_alphabet_t...,A
5,D:\df\ai\asl\asl_alphabet_train\asl_alphabet_t...,A
6,D:\df\ai\asl\asl_alphabet_train\asl_alphabet_t...,A
7,D:\df\ai\asl\asl_alphabet_train\asl_alphabet_t...,A
8,D:\df\ai\asl\asl_alphabet_train\asl_alphabet_t...,A
9,D:\df\ai\asl\asl_alphabet_train\asl_alphabet_t...,A


## datagen

In [4]:
datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valgen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


train_data=datagen.flow_from_dataframe(
    trainframe,
    target_size=(IMAGE_WIDTH,IMAGE_WIDTH),
    shuffle=True,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    color_mode="rgb",
    x_col="images",
    y_col="output",
)
test_data=valgen.flow_from_dataframe(
    testframe,
    target_size=(IMAGE_WIDTH,IMAGE_HEIGHT),
    shuffle=True,
    batch_size=1,
    class_mode='categorical',
    color_mode="rgb",
    x_col="images",
    y_col="output",
)


Found 87000 validated image filenames belonging to 29 classes.
Found 28 validated image filenames belonging to 28 classes.


# Build Model

In [5]:
import tensorflow_hub as hub
url="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
download_model = hub.KerasLayer(url,input_shape=(IMAGE_WIDTH,IMAGE_WIDTH,3))
model=keras.Sequential()
model.add(download_model)
model.add(layers.Dense(29,activation="softmax"))
model.compile(optimizers.Adam(LR),loss=losses.CategoricalCrossentropy(False),metrics=["acc"])


# Training

In [ ]:
with tf.device('/gPU:0'):
    model.fit(train_data,steps_per_epoch=train_data.samples // BATCH_SIZE,epochs=NUM_EPOCHS )

In [52]:
from sklearn.metrics import accuracy_score
y_preds=[]
y_real=[]
i=0
for ds in test_data:
    if i>28:
        break
    image,classe=enumerate(ds)
    image=image[1]
    pred=model.predict(image,verbose="0")
    y_preds.append(np.argmax(pred))
    y_real.append(np.argmax(classe[1]))
    i+=1
print("accuracy_score:",accuracy_score(y_real,y_preds))

accuracy_score: 0.896551724137931


In [56]:
model.save("model/")

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets
